In [1]:
import re
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import collections

In [2]:
import scrapy
from scrapy.selector import Selector

In [ ]:
url='https://sebenarnya.my/dakwaan-kerajaan-usir-migran-indonesia-secara-besar-besaran-sepanjang-tempoh-pkp-adalah-tidak-benar/'
page = requests.get(url)

In [32]:
soup = BeautifulSoup(page.text, 'html.parser')
x = soup.find('div',{'class':'td-post-content'})
cleanLine = [line.strip() for line in x.text.split('\n') if line.strip() != '']
cleanLine

['SEBENARNYA:',
 'Menurut kenyataan rasmi Menteri Luar:',
 'Kenyataan Migrant Care (Perhimpunan Indonesia Untuk Buruh Migran Berdaulat) yang mendakwa kononnya Kerajaan Malaysia mengusir rakyat Indonesia secara besar-besaran sepanjang tempoh Perintah Kawalan Pergerakan (PKP) adalah tidak benar dan tidak bertanggungjawab.',
 'Kenyataan ini bukan sahaja disangkal oleh Malaysia, malah turut dinafikan pihak berkepentingan Indonesia sendiri;',
 'i. Menteri Luar Indonesia, Ibu Retno Marsudi sebelum ini menyatakan memang akan ada sekumpulan rakyat Indonesia pulang, tetapi beliau tidak menyebut kerana pengusiran.',
 'ii. Jurucakap Kementerian Luar Indonesia, Teuku Faizasyah menegaskan rakyat Indonesia pulang bukan kerana diusir sebaliknya dihantar pulang kerana masuk ke Malaysia tanpa izin sebelum krisis Covid-19.',
 'iii. Kedutaan Besar Indonesia di Kuala Lumpur turut menafikan berlakunya pengusiran.',
 'iv. Koordinator Fungsi Pensosbud Konsulat Jenderal Republik Indonesia di Johor Bahru, Anan

In [58]:
#with audio
https://sebenarnya.my/dakwaan-nota-suara-anggota-atm-pukul-individu-yang-keluar-rumah-ketika-pkp-adalah-palsu/
    
#with caption
https://sebenarnya.my/kilang-assb-di-taman-perindustrian-malaysia-china-kuantan-mckip-terbakar/

786

In [77]:
# 'img_src': response.css('div.td-post-content img::attr(src)').getall(),
import pymongo

client = pymongo.MongoClient()
db = client["news"]
coll = db["sebenarnya_v1"]


In [ ]:
list(coll.find({'title':'4 Nota Suara Yang Menggunakan Nama MKN Dakwa Darurat Akan Diisytihar Adalah Palsu'}))[0]

In [99]:
db_dict = list(coll.find({'title':'4 Nota Suara Yang Menggunakan Nama MKN Dakwa Darurat Akan Diisytihar Adalah Palsu'}))[0]

title = db_dict['title']
content = db_dict['content_html']
soup = BeautifulSoup(content[0], 'html.parser')
rm_index = soup.text.find(soup.find('div',{'class':'awac-wrapper'}).text) if soup.find('div',{'class':'awac-wrapper'}) else len(soup.text)
all_text = soup.text[:rm_index]
lines = [line.strip() for line in all_text.split('\n') if line.strip() != '']

In [100]:
db_dict.keys()

dict_keys(['_id', 'date', 'title', 'content_html', 'url', 'label', 'confidence', 'category', 'meta_full_html', 'dep_content_text', 'dep_img_src'])

In [101]:
 db_dict['dep_img_src']

['https://i0.wp.com/sebenarnya.my/wp-content/uploads/2020/03/total-lockdown-from-MKN.png?fit=640%2C360&ssl=1']

In [57]:
cls = Processing(db_dict)
print(cls.title)
print(cls.label)
print(cls.category)
print(cls.content_text)
print(cls.title)
print(cls.title)
print(cls.title)

Stok Beras Negara Hanya Mampu Bertahan Selama 2.5 Bulan Adalah Tidak Benar
1
COVID-19
{'SEBENARNYA:': '', 'Menurut kenyataan rasmi Menteri Pertanian dan Industri Makanan:': 'Kementerian ingin menafikan laporan media yang menyatakan bahawa jumlah stok beras negara hanya mampu bertahan selaman 2.5 bulan.\nStok semasa beras tempatan yang berada di peringkat kilang, pemborong dan peruncit adalah sebanyak 523,000 tan metrik. Ini tidak mengambil kira stok pengeluaran beras tempatan dan beras import yang berada dalam negara.\nBuat masa ini, import beras juga tidak terjejas dan berjalan seperti sedia kala. Selain negara Vietnam, beras turut diimport dari negara-negara pengeluar lain seperti Thailand, Pakistan, Myanmar, India, Cambodia dan lain-lain. Syarikat Padiberas Nasional Bhd (BERNAS) juga diarahkan oleh Kerajaan untuk memastikan dan mempertingkatkan bekalan beras import dalam negara. Walau bagaimanapun, kebergantungan kepada beras import hanya 30% daripada keseluruhan keperluan dalam neg

In [94]:
class Processing:
    def __init__(self, db_dict:dict):
        '''take input of dict retrieved from mongoDB
        '''
        self.raw = db_dict 
        self.date = db_dict['date']
        self.url =  db_dict['url']
        self.title = self.raw['title']
        self.category = 'COVID-19'
        self.content_text, self.content_lines, self.content_html = self.parse_content()
        self.label_map = self.get_label_map()
        self.label, self.confidence = self.get_label_n_confidence()
        self.audio_srcs, self.image_scrs = self.get_figures()
        
    def get_label_map(self):
        label_map = {
            '1' : ['tidak benar(:|.|$)', 'palsu(:|.|$)'],
            '2' : ['^waspada'],
            '3' : ['penjelasan(:|.|$)', '^makluman', '\smenjelaskan\s'],
        }
        return label_map
        
        
    def parse_content(self):
        content = self.raw['content_html']
        soup = BeautifulSoup(content[0], 'html.parser')
        rm_index = (soup.text.find(soup.find('div',{'class':'awac-wrapper'}).text) 
                    if soup.find('div',{'class':'awac-wrapper'}) else len(soup.text))
        all_text = soup.text[:rm_index]
        lines = [line.strip() for line in all_text.split('\n') if line.strip() != '']

        r = re.compile(".*:$")
        keys = list(filter(r.match, lines))
        keys_gen = iter(keys)
        text_dict = {}
        old_key_index = 0
        for i, key in enumerate(keys):
            try:
                new_key_index = lines.index(next(keys_gen))
            except Exception as e:
                new_key_index = len(keys)+1
            if i == 0:
                if new_key_index == 0:
                    try:
                        new_key_index = lines.index(next(keys_gen))
                    except Exception as e:
                        new_key_index = len(keys)+1
                    text_dict[key] = '\n'.join(lines[1:new_key_index])
                else:
                    text_dict['free_text'] = '\n'.join(lines[:new_key_index])
                    try:
                        new_key_index = lines.index(next(keys_gen))
                    except Exception as e:
                        new_key_index = len(keys)+1
            else:
                text_dict[key] = '\n'.join(lines[old_key_index+1:new_key_index])
                try:
                    new_key_index = lines.index(next(keys_gen))
                except Exception as e:
                    new_key_index = len(keys)+1
            old_key_index = new_key_index

        text_dict[key] = '\n'.join(lines[old_key_index+1:])

        return text_dict, lines, soup
    
    def get_label_n_confidence(self):
        keyword_found = []
        for key in self.label_map.keys():
            for regex in self.label_map[key]:
                if re.search(regex, self.title.lower()):
                    keyword_found.append(key)
                for line in self.content_lines:
                    if re.search(regex, line):
                        keyword_found.append(key)

        if len(keyword_found)==0:
            label = 1    # default label is 1
            confidence = 3 # if nothing is found give lowest confidence
        counter = collections.Counter(keyword_found)
        label = int(counter.most_common(1)[0][0])
        confidence = 1 if len(np.unique(keyword_found))==1 else 2
        return (label, confidence)
    
    def get_figures(self):
        audio_srcs = []
        image_srcs = []
        for figure in soup.find_all('figure'):
            if figure.find('audio') is not None:
                audio_srcs.append(figure.find('audio').get('src'))
            if figure.find('img') is not None:
                image_srcs.append(figure.find('img').get('src'))
        return audio_srcs, image_scrs


In [58]:
for img_block in soup.find_all('img'):

    img_block.get('src')

In [59]:
soup.find_all('img')

[<img alt="" class="wp-image-62414" sizes="(max-width: 640px) 100vw, 640px" src="https://i0.wp.com/sebenarnya.my/wp-content/uploads/2020/03/stok-beras.png?fit=640%2C360&amp;ssl=1" srcset="https://i0.wp.com/sebenarnya.my/wp-content/uploads/2020/03/stok-beras.png?w=1920&amp;ssl=1 1920w, https://i0.wp.com/sebenarnya.my/wp-content/uploads/2020/03/stok-beras.png?resize=300%2C169&amp;ssl=1 300w, https://i0.wp.com/sebenarnya.my/wp-content/uploads/2020/03/stok-beras.png?resize=1024%2C576&amp;ssl=1 1024w, https://i0.wp.com/sebenarnya.my/wp-content/uploads/2020/03/stok-beras.png?resize=249%2C140&amp;ssl=1 249w, https://i0.wp.com/sebenarnya.my/wp-content/uploads/2020/03/stok-beras.png?resize=768%2C432&amp;ssl=1 768w, https://i0.wp.com/sebenarnya.my/wp-content/uploads/2020/03/stok-beras.png?resize=1536%2C864&amp;ssl=1 1536w, https://i0.wp.com/sebenarnya.my/wp-content/uploads/2020/03/stok-beras.png?resize=747%2C420&amp;ssl=1 747w, https://i0.wp.com/sebenarnya.my/wp-content/uploads/2020/03/stok-bera

In [102]:
audio_srcs = []
image_srcs = []
for figure in soup.find_all('figure'):
    if figure.find('audio') is not None:
        audio.append(
            figure.find('audio').get('src'))
        audio_cap
    if figure.find('img') is not None:
        image_srcs.append(figure.find('img').get('src'))
        
    

In [91]:
audio_srcs

['https://sebenarnya.my/wp-content/uploads/2020/03/WhatsApp-Audio-2020-03-28-Dr-Christopher-Lee.ogg']

In [93]:
image_srcs

['https://i0.wp.com/sebenarnya.my/wp-content/uploads/2020/03/chris-lee-1.png?fit=640%2C360&ssl=1']